In [47]:
# generate lane_idx for TuSimple dataset
import json
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np

gt_parent = "/home/automan/wuguanjie/TuSimple/train/label_data_0601.json"
saving_path = "/home/automan/wuguanjie/TuSimple/train/label_data_0601_new.json"
img_parent = "/home/automan/wuguanjie/TuSimple/test"
with open(gt_parent, "r") as f:
    old_data = f.readlines()

dump_to_json = []

for line in old_data:
    old_dict = json.loads(line)
    slope = []

    old_x_coor = old_dict["lanes"]
    y_coor = old_dict["h_samples"]
    
    #zip the coordinates
    old_coor_list = [list(zip(i, y_coor)) for i in old_x_coor]
    filtered_coor_list = []
    #calculate slope
    for i in range(len(old_coor_list)):
        coor = old_coor_list[i]
        last_idx = -1
        first_idx = 0
        while coor[last_idx][0] == -2 and last_idx > -len(coor):
            last_idx -= 1
        while coor[first_idx][0] == -2 and first_idx < len(coor) - 1:
            first_idx += 1
        if last_idx == -len(coor) or first_idx == (len(coor) -1):
            continue
        filtered_coor_list.append(coor)
        slope.append(np.arctan2(coor[last_idx][1] - coor[first_idx][1], coor[first_idx][0] - coor[last_idx][0])/ np.pi* 180)
    filtered_coor_list = [filtered_coor_list[i] for i in np.argsort(slope)]
    slope = [slope[i] for i in np.argsort(slope)]

    #create new coor_list
    new_coor_list = []
    idx_list = [None, None, None, None]
    for i in range(len(slope)):
        if slope[i] <= 90:
            idx_list[1] = i
            idx_list[0] = i - 1 if i > 0 else None
        else:
            idx_list[2] = i
            idx_list[3] = i + 1 if i + 1 < len(slope) else None
            break
    new_idx_list = []
    for i in range(len(idx_list)):
        idx = idx_list[i]
        if idx is not None:
            new_coor_list.append(filtered_coor_list[idx])
            new_idx_list.append(i+1)
    
    # seperate x and y coordinates
    lanes = []
    for i in new_coor_list:
        x, _ = zip(*i)
        lanes.append(x)
    
    old_dict["lanes"] = lanes
    old_dict["lane_idx"] = new_idx_list
    dump_to_json.append(json.dumps(old_dict))
    break

with open(saving_path,"w") as f:
    for line in dump_to_json:
        print(line, end="\n", file=f)

420 270
567 35
710 260
646 87
700 270
690 1178
420 260
721 1257
